### 数据来源
1、第三方数据接口<br>
    .http://tushare.org/trading.html#id3<br>
2、证券网站的交易明细数据<br>
    . http://quote.eastmoney.com/f1.html?code=600728&market=1<br>
    . http://vip.stock.finance.sina.com.cn/quotes_service/view/vMS_tradedetail.php?symbol=sz300044&date=2017-05-12&page=1

In [8]:
#coding=utf-8
import tushare as ts   #需要lxml等包的支持
import talib as ta
import os,time,sys,re,datetime
import pandas as pd
import numpy as np
import datetime as dt

#我的本地数据库
#import pymysql
#try:
#    conn = pymysql.connect(host="localhost", user="root",password="",database="test",charset="utf8")
#except Exception as e:
#DBfile = r".\data\stock.mdb"  # 数据库文件  
try:
    conn = pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\data\stock.mdb')
    cur = conn.cursor()
except Exception as e:
    print('engine err!')   
    print(e)

In [9]:
pro = ts.pro_api('6cbb7738f458c6366abd8601dc143cf493649fb97a7e6b4f43966902')

#### 网络数据库postgresql的使用

In [20]:
#1、首先是获取沪深两市的股票列表


    
def Get_Stock_List():
    df = ts.get_stock_basics()
    df = df.sort_index() 
    #先只筛选某行业数据
    df_hotel_and_catering=df[df['industry']=='酒店餐饮']
    df_telecom_operation=df[df['industry']=='电信运营']
    return df_hotel_and_catering,df_telecom_operation
          
#2）、获取1年交易数据，并保存
def Get_Stock_Trade(code):
    df = None
    try:
        df = ts.get_hist_data(code, start='2019-01-01', retry_count=5, pause=0.01)
        #df = pro.daily(ts_code=Code, start_date='20190401')
    except Exception as e:
        print(e,'err!')
    return df

# 3)获取构造均价、均量、MACD数据
def Make_Full_stock(code):
    df_hotel_and_catering=None
    df_telecom_operation=None
    #df =None
    try:
        df = Get_Stock_Trade(code)
    except Exception as e:
        print(code + 'trade err!')
        return df
 
    if not(df is None):
    
        # 构造成交价和成交量均线,根据具体情况构造n日均价，n日均线
        dfma25 = df['close'].rolling(min_periods=1, window=25, center=False).mean()   # 25日均价
        dfvol60 =df['volume'].rolling(min_periods=1, window=60, center=False).mean()  # 60日均量

        
        # 构造MACD值，psmacd DIFF；psmacdsignal DEA；psmacdhist DIFF-DEA
        # http://www.lfd.uci.edu/~gohlke/pythonlibs/ 下载安装 TA-Lib
        psmacd, psmacdsignal, psmacdhist = ta.MACD(np.array(df['close']), fastperiod=12, slowperiod=26, signalperiod=9)
        #转换为series
        
        #在后面增加5列，分别是14-18列，对应的是 ma25 vol60 DIFF  DEA  DIFF-DEA       
        df['ma25']=pd.Series(dfma25,index=df.index)             #ma25
        df['v_ma60']=pd.Series(dfvol60,index=df.index)          #vol60
        df['macd']=pd.Series(psmacd,index=df.index)             #DIFF
        df['macdsignal']=pd.Series(psmacdsignal,index=df.index) #DEA
        df['macdhist']=pd.Series(psmacdhist,index=df.index)     #DIFF-DEA
        
    return df

#4) 构造增加数据的Insert语句，以便将处理过某标的交易数据加入数据库
def Make_Sql(df,code):
    sqls = []

    for tdate,row in df.iterrows():    #iter迭代的应用，要支持迭代
        #Open,high,low,close,volume,amount,,,,,, = row[1:n]
       
        sql = "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5) \
        values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%.3f','%.3f','%.3f','%d')" \
        % (code, tdate, row['open'], row['high'], row['low'],row['close'],row['volume'],\
           row['ma5'],row['ma10'],row['ma20'],row['v_ma5'])
        new_sql = "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,ma25,vol5,vol60,macd,macdsignal,macdhist) \
        values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%.3f','%.3f','%.3f','%.3f','%d','%d','%.6f','%.6f','%.6f')" \
        % (code, tdate, row['open'], row['high'], row['low'],row['close'],row['volume'],\
           row['ma5'],row['ma10'],row['ma20'],row['ma25'],row['v_ma5'],row['v_ma60'],row['macd'],row['macdsignal'],row['macdhist'])
        #sqls.append(sql)
        sqls.append(new_sql)
    return sqls


# 5) 将处理过某标的交易数据加入数据库
def Insert_Stock_Trade(sqls, engine):

    for insert_sql in sqls:
        engine.execute(insert_sql)
        #print(insert_sql)
    engine.commit()
    print('Insert data over!')
   

#6) 构造各函数调用逻辑，完成程序功能
def Output_DB():
    df1 = pd.read_csv("data/电信运营2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

    df2 = pd.read_csv("data/酒店餐饮2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

    df = pd.concat([df1, df2]) #上下合并两个df

    for code in df.code:
        code = (str(code)).zfill(6)
        print("开始处理：", code)
        #print(type(code))
        df1 = Make_Full_stock(code)#获取构造均价、均量、MACD数据 后面四列数据
        print(df1.head(100))
        df1 = df1.fillna(0)
        sqls=Make_Sql(df1,code)
        print(sqls[10])
        Insert_Stock_Trade(sqls, cur)

Output_DB()





开始处理： 002467
            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  5.22  5.22   5.19  5.15  126164.84         -0.02     -0.38  5.242   
2019-11-28  5.27  5.27   5.21  5.20  138587.20         -0.04     -0.76  5.272   
2019-11-27  5.29  5.30   5.25  5.23  129196.56         -0.04     -0.76  5.324   
2019-11-26  5.27  5.33   5.29  5.26  115019.94          0.02      0.38  5.378   
2019-11-25  5.33  5.33   5.27  5.23  201445.41         -0.07     -1.31  5.400   
2019-11-22  5.45  5.51   5.34  5.32  292742.28         -0.13     -2.38  5.404   
2019-11-21  5.51  5.55   5.47  5.40  325961.69         -0.05     -0.91  5.410   
2019-11-20  5.41  5.71   5.52  5.36  625632.75          0.12      2.22  5.376   
2019-11-19  5.30  5.49   5.40  5.28  249023.14          0.11      2.08  5.334   
2019-11-18  5.28  5.29   5.29  5.20  205236.69         -0.08     -1.49  5.330   
2019-11-15  5.2

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  35.90  36.50  36.50  35.20   19683.38          0.35      0.97   
2019-11-28  36.26  36.55  36.15  35.88   15550.40          0.15      0.42   
2019-11-27  36.03  36.51  36.00  35.76   18922.62          0.11      0.31   
2019-11-26  36.11  36.78  35.89  35.60   19295.56         -0.32     -0.88   
2019-11-25  36.88  37.19  36.21  35.48   24627.20         -0.57     -1.55   
2019-11-22  37.80  38.00  36.78  36.51   26844.57         -0.78     -2.08   
2019-11-21  37.53  38.19  37.56  37.23   22267.09         -0.32     -0.84   
2019-11-20  37.52  39.22  37.88  37.47   50458.82          0.47      1.26   
2019-11-19  37.30  37.62  37.41  37.09   26114.25          0.11      0.29   
2019-11-18  36.71  37.68  37.30  36.05   32866.65          0.65      1.77   
2019-11-15  37.00  37.35  36.65  36.36   27165.49         -0.18     -0.49   

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  19.04  19.28  19.15  18.81  248652.12          0.10      0.53   
2019-11-28  19.20  19.39  19.05  19.00  194834.48         -0.25     -1.29   
2019-11-27  19.06  19.58  19.30  18.92  310695.81          0.27      1.42   
2019-11-26  18.57  19.39  19.03  18.43  481370.16          0.53      2.87   
2019-11-25  18.77  18.83  18.50  18.10  319529.47         -0.35     -1.86   
2019-11-22  19.07  19.42  18.85  18.61  303475.88         -0.18     -0.95   
2019-11-21  18.75  19.19  19.03  18.72  249640.16          0.16      0.85   
2019-11-20  18.85  19.44  18.87  18.80  361668.03          0.09      0.48   
2019-11-19  18.58  18.80  18.78  18.48  299192.97          0.18      0.97   
2019-11-18  18.12  18.67  18.60  17.88  317128.22          0.38      2.09   
2019-11-15  18.28  18.56  18.22  18.08  212014.52         -0.03     -0.16   

            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  7.84  7.86   7.82  7.73   32454.29         -0.03     -0.38  7.880   
2019-11-28  7.88  7.94   7.85  7.84   28277.65         -0.02     -0.25  7.938   
2019-11-27  7.88  7.94   7.87  7.84   43935.27          0.02      0.26  8.004   
2019-11-26  8.04  8.09   7.85  7.82   70224.00         -0.16     -2.00  8.072   
2019-11-25  8.08  8.09   8.01  7.98   55404.59         -0.10     -1.23  8.148   
2019-11-22  8.17  8.34   8.11  8.08   60726.61         -0.07     -0.86  8.166   
2019-11-21  8.16  8.23   8.18  8.13   39318.24         -0.03     -0.36  8.154   
2019-11-20  8.23  8.36   8.21  8.19   55131.14         -0.02     -0.24  8.154   
2019-11-19  8.10  8.25   8.23  8.10   56158.47          0.13      1.60  8.146   
2019-11-18  8.05  8.14   8.10  7.95   36662.16          0.05      0.62  8.140   
2019-11-15  8.18  8.20   8.0

            open  high  close   low      volume  price_change  p_change  \
date                                                                      
2019-11-29  5.70  5.73   5.71  5.68   429626.38         -0.02     -0.35   
2019-11-28  5.77  5.79   5.73  5.69   654422.75         -0.05     -0.86   
2019-11-27  5.79  5.83   5.78  5.77   438342.12         -0.05     -0.86   
2019-11-26  5.80  5.86   5.83  5.80   876359.31          0.05      0.86   
2019-11-25  5.80  5.82   5.78  5.73   555073.75         -0.01     -0.17   
2019-11-22  5.84  5.88   5.79  5.76   558183.81         -0.04     -0.69   
2019-11-21  5.82  5.86   5.83  5.78   417879.47         -0.02     -0.34   
2019-11-20  5.85  5.92   5.85  5.83   484863.75         -0.03     -0.51   
2019-11-19  5.79  5.88   5.88  5.76   601978.19          0.09      1.55   
2019-11-18  5.75  5.81   5.79  5.75   485559.22          0.04      0.70   
2019-11-15  5.85  5.85   5.75  5.74   802222.50         -0.10     -1.71   
2019-11-14  5.84  5.87   

...

            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  3.85  3.92   3.81  3.76   65117.36         -0.09     -2.31  3.800   
2019-11-28  3.75  4.11   3.90  3.71  121367.36          0.16      4.28  3.786   
2019-11-27  3.77  3.78   3.74  3.72   19561.28         -0.05     -1.32  3.752   
2019-11-26  3.76  3.80   3.79  3.73   20595.20          0.03      0.80  3.748   
2019-11-25  3.74  3.77   3.76  3.69   18626.80          0.02      0.54  3.738   
2019-11-22  3.73  3.80   3.74  3.73   24925.80          0.01      0.27  3.714   
2019-11-21  3.72  3.76   3.73  3.70   19475.00          0.01      0.27  3.696   
2019-11-20  3.74  3.78   3.72  3.70   21023.00         -0.02     -0.54  3.694   
2019-11-19  3.65  3.75   3.74  3.61   33352.36          0.10      2.75  3.692   
2019-11-18  3.69  3.69   3.64  3.61   18292.76         -0.01     -0.27  3.694   
2019-11-15  3.73  3.73   3.6

             open   high  close    low    volume  price_change  p_change  \
date                                                                       
2019-11-29   8.72   8.79   8.79   8.70   3765.55          0.02      0.23   
2019-11-28   8.77   8.78   8.77   8.70   4504.88          0.02      0.23   
2019-11-27   8.85   8.85   8.75   8.72   9895.00         -0.12     -1.35   
2019-11-26   8.92   9.00   8.87   8.80  17898.00         -0.03     -0.34   
2019-11-25   8.91   8.93   8.90   8.84   4550.64          0.00      0.00   
2019-11-22   8.93   9.02   8.90   8.86   5762.39          0.00      0.00   
2019-11-21   8.99   8.99   8.90   8.88   5400.10         -0.10     -1.11   
2019-11-20   9.05   9.10   9.00   8.98  10398.00         -0.05     -0.55   
2019-11-19   9.01   9.11   9.05   8.99   9807.80          0.00      0.00   
2019-11-18   8.99   9.05   9.05   8.86   9446.19         -0.05     -0.55   
2019-11-15   9.19   9.22   9.10   8.99   8967.67         -0.12     -1.30   
2019-11-14  

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  17.67  18.05  17.78  17.52   46401.84          0.07      0.40   
2019-11-28  17.55  18.13  17.71  17.55   69348.35          0.03      0.17   
2019-11-27  17.39  18.15  17.68  17.37   74941.83          0.37      2.14   
2019-11-26  17.71  17.79  17.31  17.31   61491.62         -0.44     -2.48   
2019-11-25  17.38  17.80  17.75  16.88   72704.92          0.25      1.43   
2019-11-22  17.06  17.67  17.50  17.05   97593.19          0.52      3.06   
2019-11-21  17.28  17.34  16.98  16.90   33035.72         -0.31     -1.79   
2019-11-20  17.67  17.70  17.29  17.15   43954.15         -0.28     -1.59   
2019-11-19  17.55  17.75  17.57  17.36   41599.66          0.02      0.11   
2019-11-18  17.33  17.65  17.55  16.96   64744.45          0.28      1.62   
2019-11-15  17.94  17.98  17.27  17.27   61449.25         -0.64     -3.57   

            open  high  close   low    volume  price_change  p_change    ma5  \
date                                                                           
2019-11-29  6.96  6.98   6.97  6.91   4930.78          0.01      0.14  6.990   
2019-11-28  6.99  7.05   6.96  6.95   6360.01         -0.04     -0.57  7.000   
2019-11-27  7.06  7.06   7.00  6.97   6097.00         -0.02     -0.28  7.020   
2019-11-26  7.04  7.07   7.02  7.01   8302.78          0.02      0.29  7.028   
2019-11-25  7.04  7.06   7.00  6.87  10439.24         -0.02     -0.28  7.056   
2019-11-22  7.09  7.14   7.02  6.98  11805.00         -0.04     -0.57  7.066   
2019-11-21  7.06  7.11   7.06  7.00   7575.00          0.02      0.28  7.058   
2019-11-20  7.15  7.15   7.04  7.04   6871.00         -0.12     -1.68  7.074   
2019-11-19  7.03  7.16   7.16  7.03   7679.14          0.11      1.56  7.096   
2019-11-18  6.98  7.08   7.05  6.94   9954.42          0.07      1.00  7.114   
2019-11-15  7.17  7.17   6.98  6.98  107

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29   9.92  10.06  10.01   9.83   29343.50          0.01      0.10   
2019-11-28  10.24  10.24  10.00   9.81   54505.99         -0.05     -0.50   
2019-11-27  10.05  10.29  10.05   9.96   59744.50         -0.18     -1.76   
2019-11-26   9.96  10.48  10.23   9.76  134862.50          0.27      2.71   
2019-11-25  10.50  10.51   9.96   9.32  188688.48         -0.40     -3.86   
2019-11-22  10.27  10.66  10.36  10.16  121794.99         -0.02     -0.19   
2019-11-21  10.57  10.97  10.38  10.32  235943.20         -0.38     -3.53   
2019-11-20  11.00  11.69  10.76  10.63  291059.19         -0.32     -2.89   
2019-11-19  10.98  11.35  11.08  10.92  195582.88         -0.10     -0.89   
2019-11-18  10.51  11.40  11.18  10.40  169269.77          0.24      2.19   
2019-11-15  10.50  11.30  10.94  10.34  167729.95         -0.12     -1.08   

#### 桌面数据库access的使用
https://blog.csdn.net/growth_path_/article/details/80748734

In [10]:
import pyodbc  
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

In [11]:
DBfile = r".\data\stock.mdb"  # 数据库文件  
try:
    conn = pyodbc.connect(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\data\stock.mdb')
    cur = conn.cursor()
except Exception as e:
     print('engine err!')


In [12]:
#创建数据库表   可以在access中创建，也可用cur.execute('CREATE TABLE xxx()')来创建 

In [13]:
sql='create table _trade( id Counter not null, code Text(6), name Text(10), trade_date Time, open Single, high Single, close Single, low Single, volume Long, amount Long,ma5 Single,ma10 Single,ma20 Single,ma25 Single,vol5 Long,vol60 Long, macd Single, macdsignal Single, macdhist Single, primary key(id) )'

#改变df后的新表 多了5列
#cur.execute(sql)
#cur.execute("drop table _trade")
#cur.execute("delete from _trade")
#cur.execute("insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5)  values('002467','2019-11-27','5.29','5.30','5.23','5.25','129196','5.324','5.350','5.439','212873')")
#cur.execute(new_sql) 
#cur.commit()

df = pd.read_sql("select * from _trade where code = '002467'", conn)
df.head(1000)

,id,code,name,trade_date,open,high,close,low,volume,amount,ma5,ma10,ma20,ma25,vol5,vol60,macd,macdsignal,macdhist
0,19431,002467,None,2019-11-27,10.05,10.29,10.05,9.96,59744,None,10.196,10.600,9.848,10.050,148206,59744,0.000000,0.000000,0.000000
1,19432,002467,None,2019-11-26,9.96,10.48,10.23,9.76,134862,None,10.338,10.600,9.795,10.140,194469,97303,0.000000,0.000000,0.000000
2,19433,002467,None,2019-11-25,10.50,10.51,9.96,9.32,188688,None,10.508,10.491,9.736,10.080,206613,127765,0.000000,0.000000,0.000000
3,19434,002467,None,2019-11-22,10.27,10.66,10.36,10.16,121794,None,10.752,10.384,9.692,10.150,202730,126272,0.000000,0.000000,0.000000
4,19435,002467,None,2019-11-21,10.57,10.97,10.38,10.32,235943,None,10.868,10.243,9.627,10.196,211917,148206,0.000000,0.000000,0.000000
5,19436,002467,None,2019-11-20,11.00,11.69,10.76,10.63,291059,None,11.004,10.108,9.554,10.290,222510,172015,0.000000,0.000000,0.000000
6,19437,002467,None,2019-11-19,10.98,11.35,11.08,10.92,195582,None,10.862,9.938,9.475,10.403,185389,175382,0.000000,0.000000,0.000000
7,19438,002467,None,2019-11-18,10.51,11.40,11.18,10.40,169269,None,10.474,9.731,9.383,10.500,151335,174618,0.000000,0.000000,0.000000
8,19439,002467,None,2019-11-15,10.50,11.30,10.94,10.34,167729,None,10.016,9.505,9.292,10.549,120377,173852,0.000000,0.000000,0.000000
9,19440,002467,None,2019-11-14,11.06,11.06,11.06,10.30,288912,None,9.618,9.307,9.209,10.600,89874,185358,0.000000,0.000000,0.000000


In [14]:
cur.commit()

#### 自定义数据获取、处理与存储函数

In [20]:
#1、首先是获取沪深两市的股票列表


    
def Get_Stock_List():
    df = ts.get_stock_basics()
    df = df.sort_index() 
    #先只筛选某行业数据
    df_hotel_and_catering=df[df['industry']=='酒店餐饮']
    df_telecom_operation=df[df['industry']=='电信运营']
    return df_hotel_and_catering,df_telecom_operation
          
#2）、获取1年交易数据，并保存
def Get_Stock_Trade(code):
    df = None
    try:
        df = ts.get_hist_data(code, start='2019-01-01', retry_count=5, pause=0.01)
        #df = pro.daily(ts_code=Code, start_date='20190401')
    except Exception as e:
        print(e,'err!')
    return df

# 3)获取构造均价、均量、MACD数据
def Make_Full_stock(code):
    df_hotel_and_catering=None
    df_telecom_operation=None
    #df =None
    try:
        df = Get_Stock_Trade(code)
    except Exception as e:
        print(code + 'trade err!')
        return df
 
    if not(df is None):
    
        # 构造成交价和成交量均线,根据具体情况构造n日均价，n日均线
        dfma25 = df['close'].rolling(min_periods=1, window=25, center=False).mean()   # 25日均价
        dfvol60 =df['volume'].rolling(min_periods=1, window=60, center=False).mean()  # 60日均量

        
        # 构造MACD值，psmacd DIFF；psmacdsignal DEA；psmacdhist DIFF-DEA
        # http://www.lfd.uci.edu/~gohlke/pythonlibs/ 下载安装 TA-Lib
        psmacd, psmacdsignal, psmacdhist = ta.MACD(np.array(df['close']), fastperiod=12, slowperiod=26, signalperiod=9)
        #转换为series
        
        #在后面增加5列，分别是14-18列，对应的是 ma25 vol60 DIFF  DEA  DIFF-DEA       
        df['ma25']=pd.Series(dfma25,index=df.index)             #ma25
        df['v_ma60']=pd.Series(dfvol60,index=df.index)          #vol60
        df['macd']=pd.Series(psmacd,index=df.index)             #DIFF
        df['macdsignal']=pd.Series(psmacdsignal,index=df.index) #DEA
        df['macdhist']=pd.Series(psmacdhist,index=df.index)     #DIFF-DEA
        
    return df

#4) 构造增加数据的Insert语句，以便将处理过某标的交易数据加入数据库
def Make_Sql(df,code):
    sqls = []

    for tdate,row in df.iterrows():    #iter迭代的应用，要支持迭代
        #Open,high,low,close,volume,amount,,,,,, = row[1:n]
       
        sql = "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,vol5) \
        values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%.3f','%.3f','%.3f','%d')" \
        % (code, tdate, row['open'], row['high'], row['low'],row['close'],row['volume'],\
           row['ma5'],row['ma10'],row['ma20'],row['v_ma5'])
        new_sql = "insert into _trade(code,trade_date,open,high,low,close,volume,ma5,ma10,ma20,ma25,vol5,vol60,macd,macdsignal,macdhist) \
        values('%s','%s','%.2f','%.2f','%.2f','%.2f','%d','%.3f','%.3f','%.3f','%.3f','%d','%d','%.6f','%.6f','%.6f')" \
        % (code, tdate, row['open'], row['high'], row['low'],row['close'],row['volume'],\
           row['ma5'],row['ma10'],row['ma20'],row['ma25'],row['v_ma5'],row['v_ma60'],row['macd'],row['macdsignal'],row['macdhist'])
        #sqls.append(sql)
        sqls.append(new_sql)
    return sqls


# 5) 将处理过某标的交易数据加入数据库
def Insert_Stock_Trade(sqls, engine):

    for insert_sql in sqls:
        engine.execute(insert_sql)
        #print(insert_sql)
    engine.commit()
    print('Insert data over!')
   

#6) 构造各函数调用逻辑，完成程序功能
def Output_DB():
    df1 = pd.read_csv("data/电信运营2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

    df2 = pd.read_csv("data/酒店餐饮2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

    df = pd.concat([df1, df2]) #上下合并两个df

    for code in df.code:
        code = (str(code)).zfill(6)
        print("开始处理：", code)
        #print(type(code))
        df1 = Make_Full_stock(code)#获取构造均价、均量、MACD数据 后面四列数据
        print(df1.head(100))
        df1 = df1.fillna(0)
        sqls=Make_Sql(df1,code)
        print(sqls[10])
        Insert_Stock_Trade(sqls, cur)

Output_DB()





开始处理： 002467
            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  5.22  5.22   5.19  5.15  126164.84         -0.02     -0.38  5.242   
2019-11-28  5.27  5.27   5.21  5.20  138587.20         -0.04     -0.76  5.272   
2019-11-27  5.29  5.30   5.25  5.23  129196.56         -0.04     -0.76  5.324   
2019-11-26  5.27  5.33   5.29  5.26  115019.94          0.02      0.38  5.378   
2019-11-25  5.33  5.33   5.27  5.23  201445.41         -0.07     -1.31  5.400   
2019-11-22  5.45  5.51   5.34  5.32  292742.28         -0.13     -2.38  5.404   
2019-11-21  5.51  5.55   5.47  5.40  325961.69         -0.05     -0.91  5.410   
2019-11-20  5.41  5.71   5.52  5.36  625632.75          0.12      2.22  5.376   
2019-11-19  5.30  5.49   5.40  5.28  249023.14          0.11      2.08  5.334   
2019-11-18  5.28  5.29   5.29  5.20  205236.69         -0.08     -1.49  5.330   
2019-11-15  5.2

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  35.90  36.50  36.50  35.20   19683.38          0.35      0.97   
2019-11-28  36.26  36.55  36.15  35.88   15550.40          0.15      0.42   
2019-11-27  36.03  36.51  36.00  35.76   18922.62          0.11      0.31   
2019-11-26  36.11  36.78  35.89  35.60   19295.56         -0.32     -0.88   
2019-11-25  36.88  37.19  36.21  35.48   24627.20         -0.57     -1.55   
2019-11-22  37.80  38.00  36.78  36.51   26844.57         -0.78     -2.08   
2019-11-21  37.53  38.19  37.56  37.23   22267.09         -0.32     -0.84   
2019-11-20  37.52  39.22  37.88  37.47   50458.82          0.47      1.26   
2019-11-19  37.30  37.62  37.41  37.09   26114.25          0.11      0.29   
2019-11-18  36.71  37.68  37.30  36.05   32866.65          0.65      1.77   
2019-11-15  37.00  37.35  36.65  36.36   27165.49         -0.18     -0.49   

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  19.04  19.28  19.15  18.81  248652.12          0.10      0.53   
2019-11-28  19.20  19.39  19.05  19.00  194834.48         -0.25     -1.29   
2019-11-27  19.06  19.58  19.30  18.92  310695.81          0.27      1.42   
2019-11-26  18.57  19.39  19.03  18.43  481370.16          0.53      2.87   
2019-11-25  18.77  18.83  18.50  18.10  319529.47         -0.35     -1.86   
2019-11-22  19.07  19.42  18.85  18.61  303475.88         -0.18     -0.95   
2019-11-21  18.75  19.19  19.03  18.72  249640.16          0.16      0.85   
2019-11-20  18.85  19.44  18.87  18.80  361668.03          0.09      0.48   
2019-11-19  18.58  18.80  18.78  18.48  299192.97          0.18      0.97   
2019-11-18  18.12  18.67  18.60  17.88  317128.22          0.38      2.09   
2019-11-15  18.28  18.56  18.22  18.08  212014.52         -0.03     -0.16   

            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  7.84  7.86   7.82  7.73   32454.29         -0.03     -0.38  7.880   
2019-11-28  7.88  7.94   7.85  7.84   28277.65         -0.02     -0.25  7.938   
2019-11-27  7.88  7.94   7.87  7.84   43935.27          0.02      0.26  8.004   
2019-11-26  8.04  8.09   7.85  7.82   70224.00         -0.16     -2.00  8.072   
2019-11-25  8.08  8.09   8.01  7.98   55404.59         -0.10     -1.23  8.148   
2019-11-22  8.17  8.34   8.11  8.08   60726.61         -0.07     -0.86  8.166   
2019-11-21  8.16  8.23   8.18  8.13   39318.24         -0.03     -0.36  8.154   
2019-11-20  8.23  8.36   8.21  8.19   55131.14         -0.02     -0.24  8.154   
2019-11-19  8.10  8.25   8.23  8.10   56158.47          0.13      1.60  8.146   
2019-11-18  8.05  8.14   8.10  7.95   36662.16          0.05      0.62  8.140   
2019-11-15  8.18  8.20   8.0

            open  high  close   low      volume  price_change  p_change  \
date                                                                      
2019-11-29  5.70  5.73   5.71  5.68   429626.38         -0.02     -0.35   
2019-11-28  5.77  5.79   5.73  5.69   654422.75         -0.05     -0.86   
2019-11-27  5.79  5.83   5.78  5.77   438342.12         -0.05     -0.86   
2019-11-26  5.80  5.86   5.83  5.80   876359.31          0.05      0.86   
2019-11-25  5.80  5.82   5.78  5.73   555073.75         -0.01     -0.17   
2019-11-22  5.84  5.88   5.79  5.76   558183.81         -0.04     -0.69   
2019-11-21  5.82  5.86   5.83  5.78   417879.47         -0.02     -0.34   
2019-11-20  5.85  5.92   5.85  5.83   484863.75         -0.03     -0.51   
2019-11-19  5.79  5.88   5.88  5.76   601978.19          0.09      1.55   
2019-11-18  5.75  5.81   5.79  5.75   485559.22          0.04      0.70   
2019-11-15  5.85  5.85   5.75  5.74   802222.50         -0.10     -1.71   
2019-11-14  5.84  5.87   

...

            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  3.85  3.92   3.81  3.76   65117.36         -0.09     -2.31  3.800   
2019-11-28  3.75  4.11   3.90  3.71  121367.36          0.16      4.28  3.786   
2019-11-27  3.77  3.78   3.74  3.72   19561.28         -0.05     -1.32  3.752   
2019-11-26  3.76  3.80   3.79  3.73   20595.20          0.03      0.80  3.748   
2019-11-25  3.74  3.77   3.76  3.69   18626.80          0.02      0.54  3.738   
2019-11-22  3.73  3.80   3.74  3.73   24925.80          0.01      0.27  3.714   
2019-11-21  3.72  3.76   3.73  3.70   19475.00          0.01      0.27  3.696   
2019-11-20  3.74  3.78   3.72  3.70   21023.00         -0.02     -0.54  3.694   
2019-11-19  3.65  3.75   3.74  3.61   33352.36          0.10      2.75  3.692   
2019-11-18  3.69  3.69   3.64  3.61   18292.76         -0.01     -0.27  3.694   
2019-11-15  3.73  3.73   3.6

             open   high  close    low    volume  price_change  p_change  \
date                                                                       
2019-11-29   8.72   8.79   8.79   8.70   3765.55          0.02      0.23   
2019-11-28   8.77   8.78   8.77   8.70   4504.88          0.02      0.23   
2019-11-27   8.85   8.85   8.75   8.72   9895.00         -0.12     -1.35   
2019-11-26   8.92   9.00   8.87   8.80  17898.00         -0.03     -0.34   
2019-11-25   8.91   8.93   8.90   8.84   4550.64          0.00      0.00   
2019-11-22   8.93   9.02   8.90   8.86   5762.39          0.00      0.00   
2019-11-21   8.99   8.99   8.90   8.88   5400.10         -0.10     -1.11   
2019-11-20   9.05   9.10   9.00   8.98  10398.00         -0.05     -0.55   
2019-11-19   9.01   9.11   9.05   8.99   9807.80          0.00      0.00   
2019-11-18   8.99   9.05   9.05   8.86   9446.19         -0.05     -0.55   
2019-11-15   9.19   9.22   9.10   8.99   8967.67         -0.12     -1.30   
2019-11-14  

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  17.67  18.05  17.78  17.52   46401.84          0.07      0.40   
2019-11-28  17.55  18.13  17.71  17.55   69348.35          0.03      0.17   
2019-11-27  17.39  18.15  17.68  17.37   74941.83          0.37      2.14   
2019-11-26  17.71  17.79  17.31  17.31   61491.62         -0.44     -2.48   
2019-11-25  17.38  17.80  17.75  16.88   72704.92          0.25      1.43   
2019-11-22  17.06  17.67  17.50  17.05   97593.19          0.52      3.06   
2019-11-21  17.28  17.34  16.98  16.90   33035.72         -0.31     -1.79   
2019-11-20  17.67  17.70  17.29  17.15   43954.15         -0.28     -1.59   
2019-11-19  17.55  17.75  17.57  17.36   41599.66          0.02      0.11   
2019-11-18  17.33  17.65  17.55  16.96   64744.45          0.28      1.62   
2019-11-15  17.94  17.98  17.27  17.27   61449.25         -0.64     -3.57   

            open  high  close   low    volume  price_change  p_change    ma5  \
date                                                                           
2019-11-29  6.96  6.98   6.97  6.91   4930.78          0.01      0.14  6.990   
2019-11-28  6.99  7.05   6.96  6.95   6360.01         -0.04     -0.57  7.000   
2019-11-27  7.06  7.06   7.00  6.97   6097.00         -0.02     -0.28  7.020   
2019-11-26  7.04  7.07   7.02  7.01   8302.78          0.02      0.29  7.028   
2019-11-25  7.04  7.06   7.00  6.87  10439.24         -0.02     -0.28  7.056   
2019-11-22  7.09  7.14   7.02  6.98  11805.00         -0.04     -0.57  7.066   
2019-11-21  7.06  7.11   7.06  7.00   7575.00          0.02      0.28  7.058   
2019-11-20  7.15  7.15   7.04  7.04   6871.00         -0.12     -1.68  7.074   
2019-11-19  7.03  7.16   7.16  7.03   7679.14          0.11      1.56  7.096   
2019-11-18  6.98  7.08   7.05  6.94   9954.42          0.07      1.00  7.114   
2019-11-15  7.17  7.17   6.98  6.98  107

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29   9.92  10.06  10.01   9.83   29343.50          0.01      0.10   
2019-11-28  10.24  10.24  10.00   9.81   54505.99         -0.05     -0.50   
2019-11-27  10.05  10.29  10.05   9.96   59744.50         -0.18     -1.76   
2019-11-26   9.96  10.48  10.23   9.76  134862.50          0.27      2.71   
2019-11-25  10.50  10.51   9.96   9.32  188688.48         -0.40     -3.86   
2019-11-22  10.27  10.66  10.36  10.16  121794.99         -0.02     -0.19   
2019-11-21  10.57  10.97  10.38  10.32  235943.20         -0.38     -3.53   
2019-11-20  11.00  11.69  10.76  10.63  291059.19         -0.32     -2.89   
2019-11-19  10.98  11.35  11.08  10.92  195582.88         -0.10     -0.89   
2019-11-18  10.51  11.40  11.18  10.40  169269.77          0.24      2.19   
2019-11-15  10.50  11.30  10.94  10.34  167729.95         -0.12     -1.08   

#### 测试自定义函数

In [319]:
#df = Get_Stock_List()    
df_hotel_and_catering,df_telecom_operation = Get_Stock_List()  
#df.head()

df = pd.concat([df_hotel_and_catering, df_telecom_operation]) #上下合并两个行业df
#df.head(100)

In [17]:

df1 = pd.read_csv("data/电信运营2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

df2 = pd.read_csv("data/酒店餐饮2019-11-21stock.csv", encoding='gbk', 
                 header = 0,
                 names = ['code', 'name', 'industry', 'area','pe', 'outstanding', 'totals', 'totalAssets','liquidAssets', 'fixedAssets', 'reserved', 
                          'reservedPerShare', 'esp', 'bvps', 'pb', 'timeToMarket', 'undp', 'perundp', 'rev', 'profit', 'gpr', 'npr', 'holders', 'MACD'])

df = pd.concat([df1, df2]) #上下合并两个df

for code in df.code:
    
    code = (str(code)).zfill(6)
    print("开始处理：", code)
    #print(type(code))
    df1 = Make_Full_stock(code)#获取构造均价、均量、MACD数据 后面四列数据
    print(df1.head(100))
    df1 = df1.fillna(0)
    
    
    sqls=Make_Sql(df1,code)
    print(sqls[10])
    Insert_Stock_Trade(sqls, cur)
    

开始处理： 002467
            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  5.22  5.22   5.19  5.15  126164.84         -0.02     -0.38  5.242   
2019-11-28  5.27  5.27   5.21  5.20  138587.20         -0.04     -0.76  5.272   
2019-11-27  5.29  5.30   5.25  5.23  129196.56         -0.04     -0.76  5.324   
2019-11-26  5.27  5.33   5.29  5.26  115019.94          0.02      0.38  5.378   
2019-11-25  5.33  5.33   5.27  5.23  201445.41         -0.07     -1.31  5.400   
2019-11-22  5.45  5.51   5.34  5.32  292742.28         -0.13     -2.38  5.404   
2019-11-21  5.51  5.55   5.47  5.40  325961.69         -0.05     -0.91  5.410   
2019-11-20  5.41  5.71   5.52  5.36  625632.75          0.12      2.22  5.376   
2019-11-19  5.30  5.49   5.40  5.28  249023.14          0.11      2.08  5.334   
2019-11-18  5.28  5.29   5.29  5.20  205236.69         -0.08     -1.49  5.330   
2019-11-15  5.2

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  35.90  36.50  36.50  35.20   19683.38          0.35      0.97   
2019-11-28  36.26  36.55  36.15  35.88   15550.40          0.15      0.42   
2019-11-27  36.03  36.51  36.00  35.76   18922.62          0.11      0.31   
2019-11-26  36.11  36.78  35.89  35.60   19295.56         -0.32     -0.88   
2019-11-25  36.88  37.19  36.21  35.48   24627.20         -0.57     -1.55   
2019-11-22  37.80  38.00  36.78  36.51   26844.57         -0.78     -2.08   
2019-11-21  37.53  38.19  37.56  37.23   22267.09         -0.32     -0.84   
2019-11-20  37.52  39.22  37.88  37.47   50458.82          0.47      1.26   
2019-11-19  37.30  37.62  37.41  37.09   26114.25          0.11      0.29   
2019-11-18  36.71  37.68  37.30  36.05   32866.65          0.65      1.77   
2019-11-15  37.00  37.35  36.65  36.36   27165.49         -0.18     -0.49   

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  19.04  19.28  19.15  18.81  248652.12          0.10      0.53   
2019-11-28  19.20  19.39  19.05  19.00  194834.48         -0.25     -1.29   
2019-11-27  19.06  19.58  19.30  18.92  310695.81          0.27      1.42   
2019-11-26  18.57  19.39  19.03  18.43  481370.16          0.53      2.87   
2019-11-25  18.77  18.83  18.50  18.10  319529.47         -0.35     -1.86   
2019-11-22  19.07  19.42  18.85  18.61  303475.88         -0.18     -0.95   
2019-11-21  18.75  19.19  19.03  18.72  249640.16          0.16      0.85   
2019-11-20  18.85  19.44  18.87  18.80  361668.03          0.09      0.48   
2019-11-19  18.58  18.80  18.78  18.48  299192.97          0.18      0.97   
2019-11-18  18.12  18.67  18.60  17.88  317128.22          0.38      2.09   
2019-11-15  18.28  18.56  18.22  18.08  212014.52         -0.03     -0.16   

            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  7.84  7.86   7.82  7.73   32454.29         -0.03     -0.38  7.880   
2019-11-28  7.88  7.94   7.85  7.84   28277.65         -0.02     -0.25  7.938   
2019-11-27  7.88  7.94   7.87  7.84   43935.27          0.02      0.26  8.004   
2019-11-26  8.04  8.09   7.85  7.82   70224.00         -0.16     -2.00  8.072   
2019-11-25  8.08  8.09   8.01  7.98   55404.59         -0.10     -1.23  8.148   
2019-11-22  8.17  8.34   8.11  8.08   60726.61         -0.07     -0.86  8.166   
2019-11-21  8.16  8.23   8.18  8.13   39318.24         -0.03     -0.36  8.154   
2019-11-20  8.23  8.36   8.21  8.19   55131.14         -0.02     -0.24  8.154   
2019-11-19  8.10  8.25   8.23  8.10   56158.47          0.13      1.60  8.146   
2019-11-18  8.05  8.14   8.10  7.95   36662.16          0.05      0.62  8.140   
2019-11-15  8.18  8.20   8.0

            open  high  close   low      volume  price_change  p_change  \
date                                                                      
2019-11-29  5.70  5.73   5.71  5.68   429626.38         -0.02     -0.35   
2019-11-28  5.77  5.79   5.73  5.69   654422.75         -0.05     -0.86   
2019-11-27  5.79  5.83   5.78  5.77   438342.12         -0.05     -0.86   
2019-11-26  5.80  5.86   5.83  5.80   876359.31          0.05      0.86   
2019-11-25  5.80  5.82   5.78  5.73   555073.75         -0.01     -0.17   
2019-11-22  5.84  5.88   5.79  5.76   558183.81         -0.04     -0.69   
2019-11-21  5.82  5.86   5.83  5.78   417879.47         -0.02     -0.34   
2019-11-20  5.85  5.92   5.85  5.83   484863.75         -0.03     -0.51   
2019-11-19  5.79  5.88   5.88  5.76   601978.19          0.09      1.55   
2019-11-18  5.75  5.81   5.79  5.75   485559.22          0.04      0.70   
2019-11-15  5.85  5.85   5.75  5.74   802222.50         -0.10     -1.71   
2019-11-14  5.84  5.87   

...

            open  high  close   low     volume  price_change  p_change    ma5  \
date                                                                            
2019-11-29  3.85  3.92   3.81  3.76   65117.36         -0.09     -2.31  3.800   
2019-11-28  3.75  4.11   3.90  3.71  121367.36          0.16      4.28  3.786   
2019-11-27  3.77  3.78   3.74  3.72   19561.28         -0.05     -1.32  3.752   
2019-11-26  3.76  3.80   3.79  3.73   20595.20          0.03      0.80  3.748   
2019-11-25  3.74  3.77   3.76  3.69   18626.80          0.02      0.54  3.738   
2019-11-22  3.73  3.80   3.74  3.73   24925.80          0.01      0.27  3.714   
2019-11-21  3.72  3.76   3.73  3.70   19475.00          0.01      0.27  3.696   
2019-11-20  3.74  3.78   3.72  3.70   21023.00         -0.02     -0.54  3.694   
2019-11-19  3.65  3.75   3.74  3.61   33352.36          0.10      2.75  3.692   
2019-11-18  3.69  3.69   3.64  3.61   18292.76         -0.01     -0.27  3.694   
2019-11-15  3.73  3.73   3.6

             open   high  close    low    volume  price_change  p_change  \
date                                                                       
2019-11-29   8.72   8.79   8.79   8.70   3765.55          0.02      0.23   
2019-11-28   8.77   8.78   8.77   8.70   4504.88          0.02      0.23   
2019-11-27   8.85   8.85   8.75   8.72   9895.00         -0.12     -1.35   
2019-11-26   8.92   9.00   8.87   8.80  17898.00         -0.03     -0.34   
2019-11-25   8.91   8.93   8.90   8.84   4550.64          0.00      0.00   
2019-11-22   8.93   9.02   8.90   8.86   5762.39          0.00      0.00   
2019-11-21   8.99   8.99   8.90   8.88   5400.10         -0.10     -1.11   
2019-11-20   9.05   9.10   9.00   8.98  10398.00         -0.05     -0.55   
2019-11-19   9.01   9.11   9.05   8.99   9807.80          0.00      0.00   
2019-11-18   8.99   9.05   9.05   8.86   9446.19         -0.05     -0.55   
2019-11-15   9.19   9.22   9.10   8.99   8967.67         -0.12     -1.30   
2019-11-14  

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29  17.67  18.05  17.78  17.52   46401.84          0.07      0.40   
2019-11-28  17.55  18.13  17.71  17.55   69348.35          0.03      0.17   
2019-11-27  17.39  18.15  17.68  17.37   74941.83          0.37      2.14   
2019-11-26  17.71  17.79  17.31  17.31   61491.62         -0.44     -2.48   
2019-11-25  17.38  17.80  17.75  16.88   72704.92          0.25      1.43   
2019-11-22  17.06  17.67  17.50  17.05   97593.19          0.52      3.06   
2019-11-21  17.28  17.34  16.98  16.90   33035.72         -0.31     -1.79   
2019-11-20  17.67  17.70  17.29  17.15   43954.15         -0.28     -1.59   
2019-11-19  17.55  17.75  17.57  17.36   41599.66          0.02      0.11   
2019-11-18  17.33  17.65  17.55  16.96   64744.45          0.28      1.62   
2019-11-15  17.94  17.98  17.27  17.27   61449.25         -0.64     -3.57   

            open  high  close   low    volume  price_change  p_change    ma5  \
date                                                                           
2019-11-29  6.96  6.98   6.97  6.91   4930.78          0.01      0.14  6.990   
2019-11-28  6.99  7.05   6.96  6.95   6360.01         -0.04     -0.57  7.000   
2019-11-27  7.06  7.06   7.00  6.97   6097.00         -0.02     -0.28  7.020   
2019-11-26  7.04  7.07   7.02  7.01   8302.78          0.02      0.29  7.028   
2019-11-25  7.04  7.06   7.00  6.87  10439.24         -0.02     -0.28  7.056   
2019-11-22  7.09  7.14   7.02  6.98  11805.00         -0.04     -0.57  7.066   
2019-11-21  7.06  7.11   7.06  7.00   7575.00          0.02      0.28  7.058   
2019-11-20  7.15  7.15   7.04  7.04   6871.00         -0.12     -1.68  7.074   
2019-11-19  7.03  7.16   7.16  7.03   7679.14          0.11      1.56  7.096   
2019-11-18  6.98  7.08   7.05  6.94   9954.42          0.07      1.00  7.114   
2019-11-15  7.17  7.17   6.98  6.98  107

             open   high  close    low     volume  price_change  p_change  \
date                                                                        
2019-11-29   9.92  10.06  10.01   9.83   29343.50          0.01      0.10   
2019-11-28  10.24  10.24  10.00   9.81   54505.99         -0.05     -0.50   
2019-11-27  10.05  10.29  10.05   9.96   59744.50         -0.18     -1.76   
2019-11-26   9.96  10.48  10.23   9.76  134862.50          0.27      2.71   
2019-11-25  10.50  10.51   9.96   9.32  188688.48         -0.40     -3.86   
2019-11-22  10.27  10.66  10.36  10.16  121794.99         -0.02     -0.19   
2019-11-21  10.57  10.97  10.38  10.32  235943.20         -0.38     -3.53   
2019-11-20  11.00  11.69  10.76  10.63  291059.19         -0.32     -2.89   
2019-11-19  10.98  11.35  11.08  10.92  195582.88         -0.10     -0.89   
2019-11-18  10.51  11.40  11.18  10.40  169269.77          0.24      2.19   
2019-11-15  10.50  11.30  10.94  10.34  167729.95         -0.12     -1.08   

In [ ]:
df1.shape


In [ ]:
sqls=Make_Sql(df1,code)
print(sqls[:3])

In [303]:
Insert_Stock_Trade(sqls, cur)

Insert data over!


###  复权数据获取
http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/000002.phtml?year=2017&jidu=1